<font color="#138D75">**NERO Winter School training**</font> <br>
**Copyright:** (c) 2025 EUMETSAT <br>
**License:** GPL-3.0-or-later <br>
**Authors:** Andrea Meraner and Dominika Leskow-Czyżewska (EUMETSAT)
# Main script to process all satellite data sources

This script aggregates all satellite data processing scripts into one to allow the quicker and automated generation of all outputs.

In [ ]:
import os
import sys
import pyproj

pyproj.datadir.set_data_dir(os.path.join(sys.prefix, 'share', 'proj'))

In [ ]:
# start should be either :00 or :30 minutes to allow the correct retrieval of both MSG and MTG data
start_time = "2024-09-14T00:00:00"
end_time = "2024-09-20T23:59:59"

# Define the latitudes and longitudes of the bounding box
W = -8.8
S = 40.6
E = -7.4
N = 40.9

lonlat_bbox = [W, S, E, N]
lonlat_bbox_str = f"{W}, {S}, {E}, {N}"

run_name = "aveiro_penalva"

output_dir = './ref_data/'

## Imagery

Landsat 8/9 and Sentinel-2

In [ ]:
from landsat_sentinel2_img_earthdata_script import main_earthdata

composites_dict = {
    'SWIR-composite': {
        'Sentinel2': ['B12', 'B8A', 'B04'],
        'Landsat89': ['B07', 'B05', 'B04']
    }
}
main_earthdata(start_time, end_time, lonlat_bbox, output_dir, run_name, composites_dict)

Sentinel-2 (full-res but daily)

In [ ]:
from sentinel2_openeo_script import main_sentinel2

bands = ["B08", "B04", "B03", "B02", "B12"]
main_sentinel2(start_time, end_time, W, S, E, N, bands, output_dir, run_name)

Sentinel-3

In [ ]:
from sentinel3_imagery_script import main_sentinel3

main_sentinel3('OLCI', start_time, end_time, W, S, E, N, output_dir, run_name)
main_sentinel3('SLSTR_SOLAR', start_time, end_time, W,S,E,N, output_dir, run_name)
main_sentinel3('SLSTR_THERMAL', start_time, end_time, W,S,E,N, output_dir, run_name)

FCI imagery

In [ ]:
input_dir = 'satellite_data/fci_data/202409_Aveiro_Penalva/'
remote_files = True
datasets = ['nir_22', 'ir_38', 'fire_temperature']
process_RC_every_minutes = 10  # change this (to a multiple of 10) if you want to reduce the frequency of FCI steps to process

from fci_satpy_script import main_fci

main_fci(input_dir, datasets, start_time, end_time, lonlat_bbox, output_dir, run_name, remote_files, process_RC_every_minutes)

## Active Fires

MODIS and VIIRS from FIRMS

In [ ]:
from firms_script import main_firms

main_firms(start_time, end_time, lonlat_bbox, output_dir, run_name)

Sentinel-3

In [ ]:

from sentinel3_fire_script import main_sentinel3_frp

main_sentinel3_frp(start_time, end_time, lonlat_bbox_str, output_dir, run_name)

MSG FRP-PIXEL

In [ ]:
from msg_frppixel_script import main_lsasaf

main_lsasaf(start_time, end_time, lonlat_bbox, output_dir, run_name)

### Zip the outputs
At last, zip the output for easier download

In [ ]:
from tools import zip_output_folder

folder_to_zip = os.path.join(output_dir, run_name)  # or input a manual path
output_zip_filename = os.path.join(output_dir, f"{run_name}.zip")  # or input a manual name

zip_output_folder(folder_to_zip, output_zip_filename)